# Materials Project Data Extraction

In [22]:
from mp_api.client import MPRester
API_KEY ='YUgbwowwn50V8bL2u8H3u8mvIVchWLpL'

No module named 'phonopy'
No module named 'phonopy'


In [23]:

material_ids = []
material_docs = []
with MPRester(API_KEY) as mpr:
    docs =  mpr.materials.summary.search(elements=["C", "S"], 
                              num_elements =(2,2))
    for doc in docs:
        material_docs.append(doc)
        material_ids.append(doc.material_id)
        

Retrieving SummaryDoc documents:   0%|          | 0/12 [00:00<?, ?it/s]

In [27]:
def convert_to_poscar(input_text):
    lines = input_text.strip().split('\n')
    try:
        # Extract element types from a dedicated line and their quantities from the next line
        elements = lines[5].split()  # Assuming 'C S' is on the seventh line
        counts = lines[6].split()  # Assuming '1 30' is on the eighth line
        print(elements)
        # Parse the scaling factor and lattice vectors
        scaling_factor = float(lines[1].strip())
        lattice_vectors = lines[2:5]

        # Process lattice vectors
        formatted_lattice_vectors = [
            f"{float(vec.split()[0]):>20.10f} {float(vec.split()[1]):>20.10f} {float(vec.split()[2]):>20.10f}"
            for vec in lattice_vectors
        ]

        # Prepare the formatted header information
        poscar_output = [
            "POSCAR(15)",
            f"{scaling_factor:.10f}",
            *formatted_lattice_vectors,
            "   " + "   ".join(elements),
            "   " + "   ".join(counts),
            "Direct"
        ]

        # Process and format positions, which begin after the 'Direct' line in the input
        positions = lines[8:]  # This assumes positions start at line 10
        formatted_positions = [
            f"{float(pos.split()[0]):>16.10f} {float(pos.split()[1]):>16.10f} {float(pos.split()[2]):>16.10f}"
            for pos in positions if pos.strip()  # Ensure there's data on the line
        ]
        poscar_output.extend(formatted_positions)

        return "\n".join(poscar_output)

    except IndexError as e:
        return f"Error processing input: {str(e)}"
    except ValueError as e:
        return f"Error processing numbers: {str(e)}"



from pymatgen.ext.matproj import MPRester
from pymatgen.io.vasp.sets import Poscar
material_poscars = []
for material_id in material_ids:
    with MPRester(API_KEY) as mpr:
        structure = mpr.materials.get_structure_by_material_id(material_id)
        # 生成POSCAR对象
        poscar = Poscar(structure)

        # 将POSCAR对象写入文件
        poscar.write_file("poscar/POSCAR_{}.vasp".format(material_id))

        f = open("poscar/POSCAR_{}.vasp".format(material_id), 'r')
        poscar_str = f.read()
        f.close()
        poscar_converted = convert_to_poscar(poscar_str)
        material_poscars.append(poscar_converted)

        

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

['C', 'S']


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

['C', 'S']


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

...

['C', 'S']


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

['C', 'S']


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

['C', 'S']


In [57]:
from pymongo import MongoClient
import pandas as pd
from django.http import HttpResponse, JsonResponse
from django.views.decorators.http import require_http_methods
import hashlib
import json

# Connect to MongoDB
client = MongoClient(host='127.0.0.1', port=27017)  # Use localhost for a standard local MongoDB instance
db = client['materialdb']  # Database name
collection_poscar = db['poscar']  # Collection name
# Function to insert a single POSCAR entry
def insert_poscar(mid, poscar_content):
    # Compute the MD5 hash of the POSCAR content
    md5 = hashlib.md5(poscar_content.encode('utf-8')).hexdigest()
    typ = "application/octet-stream"  # Type of the file content

    # Create and insert the document
    result = collection_poscar.insert_one({
        "material_id": mid,
        "md5": md5,
        "type": typ,
        "body": poscar_content
    })
    return result.inserted_id

# Insert each POSCAR into MongoDB
inserted_ids = []
for mid, poscar_content in zip(material_ids, material_poscars):
    inserted_id = insert_poscar(mid, poscar_content)
    inserted_ids.append(inserted_id)

# Print the result of the insertion
print(f"Inserted document IDs: {inserted_ids}")




Inserted document IDs: [ObjectId('6634fd42036ac3595fa5d53b'), ObjectId('6634fd42036ac3595fa5d53c'), ObjectId('6634fd42036ac3595fa5d53d'), ObjectId('6634fd42036ac3595fa5d53e'), ObjectId('6634fd42036ac3595fa5d53f'), ObjectId('6634fd42036ac3595fa5d540'), ObjectId('6634fd42036ac3595fa5d541'), ObjectId('6634fd42036ac3595fa5d542'), ObjectId('6634fd42036ac3595fa5d543'), ObjectId('6634fd42036ac3595fa5d544'), ObjectId('6634fd42036ac3595fa5d545'), ObjectId('6634fd42036ac3595fa5d546')]


In [46]:
print(len(inserted_ids))

12


In [55]:
import pandas as pd
from bson import ObjectId

# Assuming material_docs, material_ids, and inserted_ids are already defined and populated with your data
data_rows = []

# Validate that we have materials to process and that material_ids and inserted_ids are provided
if material_docs and material_ids and inserted_ids and len(material_docs) == len(material_ids) == len(inserted_ids):
    # Generate column names from the attributes of the first document, excluding 'material_id'
    columns = [attr for attr in dir(material_docs[0]) if not attr.startswith('__') and not callable(getattr(material_docs[0], attr))]
    # Construct data rows by combining material_ids with other attributes of documents
    for material_id, doc, inserted_id in zip(material_ids, material_docs, inserted_ids):
        # Convert ObjectId to hexadecimal string
        numeric_id = str(inserted_id)  # directly converts ObjectId to its hexadecimal string
        poscar = numeric_id  # Storing as a list containing only the numeric part
        row = [material_id] + [getattr(doc, col) for col in columns if hasattr(doc, col)] + [poscar]
        data_rows.append(row)
    
    # Create DataFrame with 'material_id' as the first column and 'poscar' as the last column
    df = pd.DataFrame(data_rows, columns=['material_id'] + columns + ['poscar'])

    filename = 'materials_information.xlsx'
    df.to_excel(filename, index=False)

    print(f'Data successfully written to {filename}')
else:
    print("Check that material_docs, material_ids, and inserted_ids are properly defined and have matching lengths.")


Data successfully written to materials_information.xlsx


In [58]:
from server.app.pymongo import MongoClient
import pandas as pd

# Connect to MongoDB
client = MongoClient(host='127.0.0.1', port=27017)  # Adjust host and port as necessary
db = client['materialdb']  # Database name
collection = db['materials']  # Collection name

# Load Excel file
filename = 'materials_information.xlsx'  # Update this to the path of your Excel file
df = pd.read_excel(filename)

# Convert DataFrame to dictionary and insert into MongoDB
documents = df.to_dict(orient='records')  # Converts each row to a dictionary with column names as keys

# Insert documents into MongoDB
result = collection.insert_many(documents)

# Print the result of the insertion
print(f"Inserted {len(result.inserted_ids)} documents into MongoDB.")


Inserted 12 documents into MongoDB.


In [21]:
old ="""C1 S30
1.0
  18.5068620000000017    7.8089940000000002    0.0000000000000000
 -18.5068620000000017    7.8089940000000002    0.0000000000000000
   0.0000000000000000    0.6499130000000000   16.4418429999999987
C S
1 30
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 C
   0.5257849999999999    0.4744880000000000    0.2660380000000000 S
   0.4742150000000000    0.5255120000000000    0.7339620000000000 S
   0.4607620000000000    0.2627350000000000    0.5372990000000000 S
   0.5255120000000000    0.4742150000000000    0.7339620000000000 S
   0.5392380000000000    0.7372649999999999    0.4627010000000000 S
   0.4744880000000000    0.5257849999999999    0.2660380000000000 S
   0.2546660000000000    0.5261160000000000    0.4598930000000000 S
   0.7372649999999999    0.5392380000000000    0.4627010000000000 S
   0.7453340000000001    0.4738840000000000    0.5401070000000000 S
   0.2627350000000000    0.4607620000000000    0.5372990000000000 S
   0.4738840000000000    0.7453340000000001    0.5401070000000000 S
   0.5261160000000000    0.2546660000000000    0.4598930000000000 S
   0.0049790000000000    0.3169380000000000    0.4996920000000000 S
   0.9950210000000000    0.6830619999999999    0.5003080000000000 S
   0.3426900000000000    0.5352830000000000    0.0000910000000000 S
   0.6830619999999999    0.9950210000000000    0.5003080000000000 S
   0.6573099999999999    0.4647170000000000    0.9999090000000000 S
   0.3169380000000000    0.0049790000000000    0.4996920000000000 S
   0.4997530000000000    0.0000840000000000    0.3134800000000000 S
   0.4647170000000000    0.6573099999999999    0.9999090000000000 S
   0.5002470000000000    0.9999160000000000    0.6865200000000000 S
   0.5352830000000000    0.3426900000000000    0.0000910000000000 S
   0.9999160000000000    0.5002470000000000    0.6865200000000000 S
   0.0000840000000000    0.4997530000000000    0.3134800000000000 S
   0.9977680000000000    0.9977680000000000    0.0944910000000000 S
   0.0022320000000000    0.0022320000000000    0.9055090000000000 S
   0.9678220000000000    0.1538000000000000    0.9985140000000000 S
   0.0321780000000000    0.8462000000000000    0.0014860000000000 S
   0.1538000000000000    0.9678220000000000    0.9985140000000000 S
   0.8462000000000000    0.0321780000000000    0.0014860000000000 S
"""

new = convert_to_poscar(old)
# write new into a file
with open('new_poscar.vasp', 'w') as f:
    f.write(new)

['C', 'S']


In [ ]:
list_of_available_fields